## Import experiments from former MLflow UI


this is a workaround if mlflow does not react with too many experiments on Euler

In [1]:
import mlflow
import pandas as pd

# Read the CSV file
df = pd.read_csv("models/intermediate_rtrac5c_r0r3_le2y_cpu_t10.csv")

In [3]:
experiment_name = "euler_imported3"
mlflow.create_experiment(experiment_name)

# Get the experiment ID
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

# Define the custom order for the models
model_order = {
    "linreg": 1,
    "trac": 2,
    "xgb": 3,
    "nn_reg": 4,
    "nn_class": 5,
    "nn_corn": 6,
    "rf": 7,
}

# Iterate over the rows of the DataFrame and log each run
for _, row in df.iterrows():
    with mlflow.start_run(experiment_id=experiment_id):
        # Log the metrics, parameters, and tags from the CSV
        for key, value in row.items():
            if key.startswith("metrics."):
                metric_name = key.split(".", 1)[1]
                mlflow.log_metric(metric_name, value)
            elif key.startswith("params."):
                param_name = key.split(".", 1)[1]
                mlflow.log_param(param_name, value)
            elif key == "model":
                mlflow.log_param("model", value)
                # Log the model order as a separate parameter
                mlflow.log_param("model_order", model_order.get(value, 999))
            elif key == "time_total_s":
                mlflow.log_metric("time_total_s", value)
            elif key.startswith("tags."):
                tag_name = key.split(".", 1)[1]
                mlflow.set_tag(tag_name, value)

In [4]:
! mlflow ui --port 5004

[2024-05-05 13:41:53 +0200] [35348] [INFO] Starting gunicorn 21.2.0
[2024-05-05 13:41:53 +0200] [35348] [INFO] Listening at: http://127.0.0.1:5004 (35348)
[2024-05-05 13:41:53 +0200] [35348] [INFO] Using worker: sync
[2024-05-05 13:41:53 +0200] [35350] [INFO] Booting worker with pid: 35350
[2024-05-05 13:41:53 +0200] [35352] [INFO] Booting worker with pid: 35352
[2024-05-05 13:41:53 +0200] [35353] [INFO] Booting worker with pid: 35353
[2024-05-05 13:41:53 +0200] [35354] [INFO] Booting worker with pid: 35354
^C
[2024-05-05 13:44:47 +0200] [35348] [INFO] Handling signal: int
[2024-05-05 13:44:47 +0200] [35352] [INFO] Worker exiting (pid: 35352)
[2024-05-05 13:44:47 +0200] [35350] [INFO] Worker exiting (pid: 35350)
[2024-05-05 13:44:47 +0200] [35354] [INFO] Worker exiting (pid: 35354)
[2024-05-05 13:44:47 +0200] [35353] [INFO] Worker exiting (pid: 35353)
